In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import rushd as rd
import yaml
import scipy as sp
import numpy as np

from pathlib import Path
from typing import Any, Dict, List, Set, Tuple, Union, Optional

datadir = Path(Path('../datadir.txt').read_text())

basepath = rd.datadir/'instruments'/'data'/'qPCR'/'emma'/'command'/'2024.11.13_command'
output_path = rd.rootdir/'output'/'qPCR'/'Biorep_1'

In [ ]:
rd.plot.plot_well_metadata(basepath/'metadata.yaml')

In [ ]:
with (basepath/'metadata.yaml').open() as f:
    metadata_dict = yaml.safe_load(f)['metadata']
    metadata = pd.DataFrame({
        k: rd.well_mapper.well_mapping(v) for k,v in metadata_dict.items()
    })

In [ ]:
qpcr_cp = pd.read_csv(basepath/'2024.11.13_ELP_command_qPCR_Cp.txt', sep='\t', header=1)
df = qpcr_cp.join(metadata, on='Pos')

In [ ]:
GAPDH_means = df.groupby(['primer_set','gene','circuit','dox'])['Cp'].mean().xs('GAPDH', level='primer_set')
normed_df = pd.merge(df, GAPDH_means.rename('GAPDH_Cp'), how='inner', on=['gene','circuit','dox'])
normed_df['dCt'] = normed_df['Cp'] - normed_df['GAPDH_Cp']
normed_df['expression'] = 2.0**(-normed_df['dCt'])
display(normed_df)

In [ ]:
plot_df = df.loc[(df.circuit=='control')|(df.circuit=='no-RT')]

sns.scatterplot(data=plot_df, x='primer_set', y='Cp', hue='gene')
plt.xticks(rotation=60) 
plt.savefig(str(output_path)+'/controls_with_no_RT.svg',bbox_inches='tight')